In [1]:
import wandb
import pandas as pd

path = 'advertising.csv'

In [2]:
df = pd.read_csv(path)
print(df.head())

      TV  Radio  Newspaper  Sales
0  230.1   37.8       69.2   22.1
1   44.5   39.3       45.1   10.4
2   17.2   45.9       69.3   12.0
3  151.5   41.3       58.5   16.5
4  180.8   10.8       58.4   17.9


In [3]:
def scaling(data1, data2, data3):
    data = data1 + data2 + data3
    max_value = max(data)
    min_value = min(data)
    mean_value = sum(data) / len(data)

    data1 = [(x - mean_value) / (max_value - min_value) for x in data1]
    data2 = [(x - mean_value) / (max_value - min_value) for x in data2]
    data3 = [(x - mean_value) / (max_value - min_value) for x in data3]
    return (data1, data2, data3)

In [4]:
def predict(x1, x2, x3, w1, w2, w3, b):
    return x1*w1 + x2*w2 + x3*w3 + b

def gradient_w(y_hat, y, xi):
    dwi = 2*xi*(y_hat - y)
    return dwi

def gradient_b(y_hat, y):
    db = 2*(y_hat - y)
    return db

def update_weight_w(wi, dwi, lr):
    wi_new = wi - lr*dwi
    return wi_new

def update_weight_b(b, db, lr):
    b_new = b - lr*db
    return b_new

In [5]:
b = 1
w1, w2, w3 = 0, 0, 0
lr = 0.01
epochs = 1000

In [6]:
wandb.init(
    project="demo-linear-regression",
    config={
        "learning_rate": lr,
        "epochs": epochs,
    },
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vhp08072004. Use `wandb login --relogin` to force relogin


In [7]:
wandb.run.log({"Dataset": wandb.Table(dataframe=df)})

In [8]:
tv_data = df['TV']
radio_data = df['Radio']
newspaper_data = df['Newspaper']
(X1_train, X2_train, X3_train) = scaling(tv_data, radio_data, newspaper_data)
Y_train = df['Sales']

In [9]:
N = len(X1_train)
losses = []

In [10]:
for epoch in range(epochs):
    loss_total = 0.0
    for i in range(N):
        x1 = X1_train[i]
        x2 = X2_train[i]
        x3 = X3_train[i]
        y = Y_train[i]

        y_hat = predict(x1, x2, x3, w1, w2, w3, b)
        dw1 = gradient_w(y_hat, y, x1)
        dw2 = gradient_w(y_hat, y, x2)
        dw3 = gradient_w(y_hat, y, x3)
        db = gradient_b(y_hat, y)

        w1 = update_weight_w(w1, dw1, lr)
        w2 = update_weight_w(w2, dw2, lr)
        w3 = update_weight_w(w3, dw3, lr)
        b = update_weight_b(b, db, lr)

        loss = (y_hat - y)*(y_hat - y)/2.0
        loss_total = loss_total + loss
    wandb.log({"loss": loss_total/N})
    losses.append(loss_total/N)

wandb.finish()

loss,█▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,1.38004
